In [1]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
import paramiko
import io
pd.set_option('display.max_columns',30)

c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [48]:
import sys
print(sys.executable)

c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\python.exe


# A. Load Data

## 1. DBA: Master Funding-Master Table

In [2]:
# credentials
username = os.environ['DBA_USR']
password = os.environ['DBA_PW']
dsn = '10.220.50.121:1661/ANALYTIC'

In [3]:
# create connection
connection = oracledb.connect(user=username, password=password, dsn=dsn)

In [4]:
# create engine
engine = create_engine('oracle+oracledb://', creator=lambda: connection)

In [5]:
# open and read the file as a single buffer
mfQuery = open(r'MF.sql', 'r')
sqlFile = mfQuery.read()
mfQuery.close()

In [6]:
# run query and store data in df
MF = pd.read_sql(sqlFile, engine)

In [7]:
# change column names into UPPER CASE
MF.columns = [x.upper() for x in MF.columns]

In [8]:
# create source column for MASTER FUNDING data (TBL_BAL)
MF['SOURCE'] = 'TBL_BAL'

In [9]:
# convert BASE_DT to int for compatibility reasons
MF['BASE_DT'] = MF['BASE_DT'].astype(int)

In [10]:
MF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117464 entries, 0 to 117463
Data columns (total 41 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BASE_DT          117464 non-null  int32  
 1   BASE_YM          117464 non-null  object 
 2   AGREE_ID         117464 non-null  object 
 3   FLAG             117464 non-null  object 
 4   ACCT_NO          117464 non-null  object 
 5   ACCT_BR          117464 non-null  object 
 6   GCIF_CREATE_BR   117463 non-null  object 
 7   REGION           117462 non-null  object 
 8   AREA             117462 non-null  object 
 9   BRANCH           117462 non-null  object 
 10  GCIF_NO          117464 non-null  object 
 11  CIF_NO           117464 non-null  object 
 12  CUST_TYPE        117464 non-null  object 
 13  PROD_NM          117464 non-null  object 
 14  SUB_PROD_NM      117464 non-null  object 
 15  SEGMENT          117464 non-null  object 
 16  GCIF_NAME        117464 non-null  obje

In [11]:
MF.columns

Index(['BASE_DT', 'BASE_YM', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'ACCT_BR',
       'GCIF_CREATE_BR', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
       'CUST_TYPE', 'PROD_NM', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME',
       'PROD_TYPE', 'CURR_CODE', 'SUB_LOB_CD', 'RATE_DPK', 'BASE_AMT_FIX',
       'MTD_AVG_AMT_FIX', 'MTD_INT_EXP', 'NPK_SALES', 'COLT', 'COF_MTD', 'DTD',
       'MTD', 'YTD', 'HIGH_COF_FLAG', 'SEGMENT_FIX', 'LOB_SORT', 'CASA_TD',
       'DTD_10B', 'MTD_10B', 'DIVISION', 'STATUS', 'SYSTEM_TYPE', 'CTRL3',
       'SOURCE'],
      dtype='object')

## 2. Local Files

In [12]:
path = os.getcwd()
parDir = path

### a. RMR

In [13]:
# RMR dir
RMR_Dir = parDir+'\\RMR\\'
RMR_Files = list(filter(os.path.isfile, glob.glob(RMR_Dir + '*')))
RMR_Files.sort()

In [14]:
# check files
RMR_Files

['d:\\Work\\Process Automation\\Daily Funding\\RMR\\RMR 2408.xlsx']

In [15]:
# read file(s)
df_RMR = pd.read_excel(RMR_Files[-1])

### b. Target

In [16]:
# Target dir
Target_Dir = parDir+'\\Target\\'
Target_Files = list(filter(os.path.isfile, glob.glob(Target_Dir + '*')))
Target_Files.sort()

In [17]:
# check files                                                                   
Target_Files

['d:\\Work\\Process Automation\\Daily Funding\\Target\\Target 2409.xlsx']

In [18]:
# read file(s)
df_Target = pd.read_excel(Target_Files[-1])

# B. Processing

## 1. MF Aggregate

In [19]:
# aggregate columns
MF_Agg = MF.groupby(['BASE_DT', 'BASE_YM', 'FLAG', 'SEGMENT', 'PROD_TYPE', 'DIVISION'], dropna=False)['BASE_AMT_FIX'].agg(BASE_AMT_FIX = 'sum')
MF_Agg.reset_index(inplace=True)

In [20]:
MF_Agg

,BASE_DT,BASE_YM,FLAG,SEGMENT,PROD_TYPE,DIVISION,BASE_AMT_FIX
0,20241206,202412,Conven,CFS-NONRB-BB,CA,UNDEFINED,7.228744e+12
1,20241206,202412,Conven,CFS-NONRB-BB,SA,UNDEFINED,1.855442e+10
2,20241206,202412,Conven,CFS-NONRB-BB,TD,UNDEFINED,5.523886e+12
3,20241206,202412,Conven,CFS-NONRB-MICRO,CA,UNDEFINED,8.074488e+08
4,20241206,202412,Conven,CFS-NONRB-MICRO,SA,UNDEFINED,3.837533e+09
5,20241206,202412,Conven,CFS-NONRB-MICRO,TD,UNDEFINED,1.277719e+09
6,20241206,202412,Conven,CFS-NONRB-RSME,CA,UNDEFINED,8.471450e+12
7,20241206,202412,Conven,CFS-NONRB-RSME,SA,UNDEFINED,1.668689e+11
8,20241206,202412,Conven,CFS-NONRB-RSME,TD,UNDEFINED,4.524188e+12
9,20241206,202412,Conven,CFS-NONRB-SME+,CA,UNDEFINED,1.518326e+12


In [21]:
MF_Agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BASE_DT       42 non-null     int32  
 1   BASE_YM       42 non-null     object 
 2   FLAG          42 non-null     object 
 3   SEGMENT       42 non-null     object 
 4   PROD_TYPE     42 non-null     object 
 5   DIVISION      42 non-null     object 
 6   BASE_AMT_FIX  42 non-null     float64
dtypes: float64(1), int32(1), object(5)
memory usage: 2.3+ KB


In [22]:
MF_Agg['SOURCE'] = 'TBL_BAL_SUMMARY'

In [23]:
MF_All = pd.concat([MF,MF_Agg], ignore_index=True)
MF_All['BASE_DT'] = MF_All['BASE_DT'].astype(int)
MF_All['BASE_YM'] = MF_All['BASE_YM'].astype(int)

In [24]:
MF_All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117506 entries, 0 to 117505
Data columns (total 41 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BASE_DT          117506 non-null  int32  
 1   BASE_YM          117506 non-null  int32  
 2   AGREE_ID         117464 non-null  object 
 3   FLAG             117506 non-null  object 
 4   ACCT_NO          117464 non-null  object 
 5   ACCT_BR          117464 non-null  object 
 6   GCIF_CREATE_BR   117463 non-null  object 
 7   REGION           117462 non-null  object 
 8   AREA             117462 non-null  object 
 9   BRANCH           117462 non-null  object 
 10  GCIF_NO          117464 non-null  object 
 11  CIF_NO           117464 non-null  object 
 12  CUST_TYPE        117464 non-null  object 
 13  PROD_NM          117464 non-null  object 
 14  SUB_PROD_NM      117464 non-null  object 
 15  SEGMENT          117506 non-null  object 
 16  GCIF_NAME        117464 non-null  obje

## 2. RMR

DB data source unavailable. Requires manual file update.

In [25]:
# create RMR container with identical columns to MF
RMRContainer = MF.iloc[0:0]

In [26]:
# Tag RMR data
df_RMR['SOURCE'] = 'RMR'

In [27]:
df_RMR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BASE_YM       30 non-null     int64  
 1   FLAG          30 non-null     object 
 2   SEGMENT       30 non-null     object 
 3   PROD_TYPE     30 non-null     object 
 4   BASE_AMT_FIX  30 non-null     float64
 5   SOURCE        30 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.5+ KB


In [28]:
# Create Lob Mapping
lobMap = pd.DataFrame({'SEGMENT':["CFS-NONRB-MICRO", "CFS-NONRB-RSME", "CFS-NONRB-SME+", "CFS-NONRB-BB", "XXX-GB-CORP"],
                       'SEGMENT_FIX':["CFS-SMER & MICRO","CFS-SMER & MICRO", "CFS-SME+", "CFS-BB", "GB-CORP"]})

In [29]:
# merge LoB Mapping
df_RMR = df_RMR.merge(lobMap, on='SEGMENT', how='left')

In [30]:
# concat data to empty df
df_RMR = pd.concat([RMRContainer, df_RMR])

In [31]:
df_RMR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BASE_DT          0 non-null      float64
 1   BASE_YM          30 non-null     object 
 2   AGREE_ID         0 non-null      object 
 3   FLAG             30 non-null     object 
 4   ACCT_NO          0 non-null      object 
 5   ACCT_BR          0 non-null      object 
 6   GCIF_CREATE_BR   0 non-null      object 
 7   REGION           0 non-null      object 
 8   AREA             0 non-null      object 
 9   BRANCH           0 non-null      object 
 10  GCIF_NO          0 non-null      object 
 11  CIF_NO           0 non-null      object 
 12  CUST_TYPE        0 non-null      object 
 13  PROD_NM          0 non-null      object 
 14  SUB_PROD_NM      0 non-null      object 
 15  SEGMENT          30 non-null     object 
 16  GCIF_NAME        0 non-null      object 
 17  PROD_TYPE        3

## 3. Target

In [32]:
# assign container
TargetContainer = RMRContainer

In [33]:
# tag source data
df_Target['SOURCE'] = 'TARGET'

In [34]:
df_Target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BASE_YM       30 non-null     int64  
 1   FLAG          30 non-null     object 
 2   SEGMENT       30 non-null     object 
 3   PROD_TYPE     30 non-null     object 
 4   BASE_AMT_FIX  30 non-null     float64
 5   SOURCE        30 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.5+ KB


In [35]:
# merge to LoB Map
df_Target = df_Target.merge(lobMap, on='SEGMENT', how='left')

In [36]:
# concat data to empty df
df_Target = pd.concat([TargetContainer, df_Target])

In [37]:
df_Target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BASE_DT          0 non-null      float64
 1   BASE_YM          30 non-null     object 
 2   AGREE_ID         0 non-null      object 
 3   FLAG             30 non-null     object 
 4   ACCT_NO          0 non-null      object 
 5   ACCT_BR          0 non-null      object 
 6   GCIF_CREATE_BR   0 non-null      object 
 7   REGION           0 non-null      object 
 8   AREA             0 non-null      object 
 9   BRANCH           0 non-null      object 
 10  GCIF_NO          0 non-null      object 
 11  CIF_NO           0 non-null      object 
 12  CUST_TYPE        0 non-null      object 
 13  PROD_NM          0 non-null      object 
 14  SUB_PROD_NM      0 non-null      object 
 15  SEGMENT          30 non-null     object 
 16  GCIF_NAME        0 non-null      object 
 17  PROD_TYPE        3

# C. MASKING

In [38]:
# container for masked acct and agr
acct_fix = []
agr_fix = []

# mask acct and agr
for i in list(MF_All.index):
    agree = str(MF_All.loc[i,'AGREE_ID'])
    a = agree[:-7] + "XXXX" + agree[-3:]
    acct = str(MF_All.loc[i, "ACCT_NO"])
    b = acct[:-7] + "XXXX" + acct[-3:]
    agr_fix.append(a)
    acct_fix.append(b)

# replace original field with masked values
MF_All['AGREE_ID'] = agr_fix
MF_All['ACCT_NO'] = acct_fix

In [39]:
# name masking function
def masking_name(param):
    if param not in ["nan","NaN"]:
        return  ' '.join([item.replace(item[1:-1], "*"*len(item[1:-1])) if len(item)>2 and item[-1]!="," and item[-1]!="." else (item.replace(item[1:-2], "*"*len(item[1:-2])) if len(item)>2 and (item[-1]==",") else (item if len(item)>2 and (item[-1]==".") else item.replace(item[1:-2], "*"*len(item[1:-2])))) for item in param.split()])
    else:
        return param

In [40]:
# mask names
MF_All["GCIF_NAME"] = MF_All["GCIF_NAME"].astype(str)
MF_All["GCIF_NAME"] = MF_All["GCIF_NAME"].apply(masking_name) 

In [41]:
MF_All.columns

Index(['BASE_DT', 'BASE_YM', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'ACCT_BR',
       'GCIF_CREATE_BR', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
       'CUST_TYPE', 'PROD_NM', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME',
       'PROD_TYPE', 'CURR_CODE', 'SUB_LOB_CD', 'RATE_DPK', 'BASE_AMT_FIX',
       'MTD_AVG_AMT_FIX', 'MTD_INT_EXP', 'NPK_SALES', 'COLT', 'COF_MTD', 'DTD',
       'MTD', 'YTD', 'HIGH_COF_FLAG', 'SEGMENT_FIX', 'LOB_SORT', 'CASA_TD',
       'DTD_10B', 'MTD_10B', 'DIVISION', 'STATUS', 'SYSTEM_TYPE', 'CTRL3',
       'SOURCE'],
      dtype='object')

In [42]:
# Assign to MF as currfin with selected columns
currFin = MF_All[['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO','ACCT_BR',
                  'GCIF_CREATE_BR', 'REGION', 'AREA', 'BRANCH',
                  'GCIF_NO', 'CIF_NO', 'CUST_TYPE', 'PROD_NM','SUB_PROD_NM', 'SEGMENT',
                  'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE', 'SUB_LOB_CD','NPK_SALES', 'COLT',
                  'RATE_DPK','BASE_AMT_FIX','MTD_AVG_AMT_FIX', 'MTD_INT_EXP', 
                  'DTD', 'MTD', 'YTD', 'DIVISION', 'SOURCE',
                  'SEGMENT_FIX', 'COF_MTD', 'MTD_INT_EXP','HIGH_COF_FLAG', 
                  'LOB_SORT','CASA_TD','DTD_10B','MTD_10B']]

In [43]:
# remove possible duplicates
currFin.drop_duplicates(inplace=True)

C:\Users\MR38804\AppData\Local\Temp\ipykernel_16640\1603293702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currFin.drop_duplicates(inplace=True)


In [44]:
currFin.head()

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,ACCT_BR,GCIF_CREATE_BR,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,CUST_TYPE,PROD_NM,SUB_PROD_NM,SEGMENT,...,MTD_AVG_AMT_FIX,MTD_INT_EXP,DTD,MTD,YTD,DIVISION,SOURCE,SEGMENT_FIX,COF_MTD,MTD_INT_EXP,HIGH_COF_FLAG,LOB_SORT,CASA_TD,DTD_10B,MTD_10B
0,20241206,ST16IDR1670000116XXXX861,Conven,0000116XXXX861,167,110,JAWA BARAT,JAWA BARAT 4,KCI TASIKMALAYA,G000359935,0006642982,INDV,Maybank Tabungan Woman One,Maybank Tabungan Woman One,CFS-NONRB-RSME,...,38823120.31,0.0,0.0,0.0,7503036.00,UNDEFINED,TBL_BAL,CFS-SMER & MICRO,0.0,0.0,None,1.0,CASA,NO,NO
1,20241206,ST16IDR4013000004XXXX136,Conven,3000004XXXX136,401,016,JAKARTA 3,BEKASI,KCP HARAPAN INDAH,G000456624,0000044764,NON INDV,Maybank Deposito,Maybank Deposito,CFS-NONRB-BB,...,59310883.50,0.0,0.0,0.0,0.00,UNDEFINED,TBL_BAL,CFS-BB,0.0,0.0,None,3.0,TD,NO,NO
2,20241206,IM16IDR016201XXXX474,Conven,201XXXX474,016,016,JAKARTA 2,KELAPA GADING,KCI KELAPA GADING,G000456624,0000044764,NON INDV,Maybank Giro,Maybank Giro,CFS-NONRB-BB,...,58125.00,0.0,0.0,-105000.0,-542062.34,UNDEFINED,TBL_BAL,CFS-BB,0.0,0.0,None,3.0,CASA,NO,NO
3,20241206,ST16IDR0013002060XXXX766,Conven,3002060XXXX766,001,016,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000456624,0000044764,NON INDV,Maybank Deposito,Maybank Deposito,CFS-NONRB-BB,...,38369863.00,0.0,0.0,0.0,0.00,UNDEFINED,TBL_BAL,CFS-BB,0.0,0.0,None,3.0,TD,NO,NO
4,20241206,IM16IDR030203XXXX675,Conven,203XXXX675,121,075,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000459396,0000136071,NON INDV,Maybank Giro,Maybank Giro,CFS-NONRB-RSME,...,4919376.00,0.0,0.0,-55000.0,-649571.00,UNDEFINED,TBL_BAL,CFS-SMER & MICRO,0.0,0.0,None,1.0,CASA,NO,NO


In [45]:
# Data date
MF_date = currFin.iloc[0,0]
RMR_date = df_RMR.iloc[0,0]
Target_date = df_Target.iloc[0,0]

In [46]:
currFin.to_csv(parDir+f'\\Temp_Result\\MF_{MF_date}.csv', index=False)

In [47]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117468 entries, 0 to 117505
Data columns (total 38 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BASE_DT          117468 non-null  int32  
 1   AGREE_ID         117468 non-null  object 
 2   FLAG             117468 non-null  object 
 3   ACCT_NO          117468 non-null  object 
 4   ACCT_BR          117426 non-null  object 
 5   GCIF_CREATE_BR   117425 non-null  object 
 6   REGION           117424 non-null  object 
 7   AREA             117424 non-null  object 
 8   BRANCH           117424 non-null  object 
 9   GCIF_NO          117426 non-null  object 
 10  CIF_NO           117426 non-null  object 
 11  CUST_TYPE        117426 non-null  object 
 12  PROD_NM          117426 non-null  object 
 13  SUB_PROD_NM      117426 non-null  object 
 14  SEGMENT          117468 non-null  object 
 15  GCIF_NAME        117468 non-null  object 
 16  PROD_TYPE        117468 non-null  object 
 

# D. Save Result

## 1. Connect to SFTP

In [ ]:
# SFTP connection details
hostname = '10.220.42.38'
port = 22
username = os.environ['UID']
password = os.environ['UPW']

## 2. Save to SFTP

### a. Master Funding

In [46]:
# convert dataframe to Memory
csv_buffer = io.StringIO()
MF.to_csv(csv_buffer, index=False)
csv_buffer.seek(0) # reset buffer position to beginning

# Save path
remote_path = f'/PDA/PNR/DAILY_FUNDING/MF_{MF_date}.csv'

# Estabilish SFTP connection
try:
    # Create the SSH client
    ssh = paramiko.SSHClient()
    ssh.load_system_host_keys()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(hostname, port=port, username=username, password=password)

    # Open SFTP session
    sftp = ssh.open_sftp()

    # Write the buffer to the remote file
    with sftp.file(remote_path, 'w') as remote_file:
        remote_file.write(csv_buffer.getvalue())

    print(f'File uploaded successfully to {remote_path}')
except Exception as e:
    print(f'An error occured: {e}')
finally:
    if 'sftp' in locals():
        sftp.close()
    if 'ssh' in locals():
        ssh.close()

An error occured: Authentication failed.


### b. RMR

In [ ]:
# Load YMKeeper
YMRecordPrev = pd.read_csv(parDir+'\\Keeper\YMKeeper.csv')
RMRecordYM = YMRecordPrev.iloc[0,0]
TargetRecordYM = YMRecordPrev.iloc[0,1]

In [ ]:
if df_RMR.iloc[0,0] > RMRecordYM:
    # convert dataframe to Memory
    csv_buffer = io.StringIO()
    df_RMR.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0) # reset buffer position to beginning

    # Save path
    remote_path = f'/PDA/PNR/DAILY_FUNDING/RMR_{RMR_date}.csv'

    # Estabilish SFTP connection
    try:
        # Create the SSH client
        ssh = paramiko.SSHClient()
        ssh.load_system_host_keys()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
        ssh.connect(hostname, port=port, username=username, password=password)

        # Open SFTP session
        sftp = ssh.open_sftp()

        # Write the buffer to the remote file
        with sftp.file(remote_path, 'w') as remote_file:
            remote_file.write(csv_buffer.getvalue())

        print(f'File uploaded successfully to {remote_path}')
    except Exception as e:
        print(f'An error occured: {e}')
    finally:
        if 'sftp' in locals():
            sftp.close()
        if 'ssh' in locals():
            ssh.close()

### c. Target

In [ ]:
if df_Target.iloc[0,0] > TargetRecordYM:
    # convert dataframe to Memory
    csv_buffer = io.StringIO()
    df_Target.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0) # reset buffer position to beginning
    # Save path
    remote_path = f'/PDA/PNR/DAILY_FUNDING/Target_{Target_date}.csv'

    # Estabilish SFTP connection
    try:
        # Create the SSH client
        ssh = paramiko.SSHClient()
        ssh.load_system_host_keys()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
        ssh.connect(hostname, port=port, username=username, password=password)

        # Open SFTP session
        sftp = ssh.open_sftp()

        # Write the buffer to the remote file
        with sftp.file(remote_path, 'w') as remote_file:
            remote_file.write(csv_buffer.getvalue())

        print(f'File uploaded successfully to {remote_path}')
    except Exception as e:
        print(f'An error occured: {e}')
    finally:
        if 'sftp' in locals():
            sftp.close()
        if 'ssh' in locals():
            ssh.close()

## 3. Save YM Keeper

In [ ]:
# save RMR and Target YM
YMRecord = pd.DataFrame({'RMR_YM':df_RMR.iloc[0,0]},
                        {'Target_YM':df_Target.iloc[0,0]})
YMRecord.to_csv(parDir+'\\Keeper\YMKeeper.csv', index=False)